In [3]:
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
# import plotly.express as px
# from geopy.geocoders import Nominatim
# import time
# import plotly.graph_objects as go
# import seaborn as sns
# import matplotlib.pyplot as plt
# from ipywidgets import widgets
# from IPython.display import display, clear_output
# import ipywidgets as widgets
# from ipywidgets import interact

# Define the API endpoint and parameters
url = "https://api.fda.gov/food/enforcement.json?search=report_date:[20200101+TO+20241201]&limit=1000"

# Send a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    recalls = data.get('results', [])
    recall_df = pd.DataFrame(recalls)
   
    # Capitalize the first letter of each column name
    recall_df.columns = [col.capitalize() for col in recall_df.columns]

    # Establish a connection to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Replace with your MongoDB connection string

    # Access a database and collection
    db = client["data_fda"]  # Replace "fda_data" with your desired database name
    collection = db["food_recalls"]  # Replace "food_recalls" with your desired collection name

    collection.update_many({}, {"$unset": {"Openfda": "",
                                        "Event_id": "",
                                        "Product_type": "",
                                        "Address_2": "",
                                        "Voluntary_mandated": "",
                                        "Initial_firm_notification": "",
                                        "Distribution_pattern": "",
                                        "Recall_number": "",
                                        "Product_description": "",
                                        "Product_quantity": "",
                                        "Reason_for_recall": "",
                                        "Report_date": "",
                                        "Code_info": "",
                                        "More_code_info": ""
                                        }}          
                            )

    # Rename the column "Recall_firm" to "Recall Firm"
    collection.update_many({}, {"$rename": {"Recalling_firm": "Recall Firm",
                                            "Address_1": "Address",
                                            "Postal_code": "Postal Code",
                                            "Recall_initiation_date": "Recall Initiation Date",
                                            "Center_classification_date": "Center Classification Date",
                                            "Termination_date": "Termination Date"
                                            }}
                            )

    ##(Only Run once to format the dates)
    # Update all documents to convert 'Center Classification Date' to ISODate
    # documents = collection.find({"Center Classification Date": {"$exists": True}})
    # for doc in documents:
    #     date_str = doc["Center Classification Date"]
        
    #     # Convert string '20200413' to ISODate
    #     try:
    #         date_obj = datetime.strptime(date_str, "%Y%m%d")  # Parse the string to a date object
    #     except ValueError as e:
    #         print(f"Error converting date for document ID {doc['_id']}: {e}")
    #         continue

    #     # Update the document with the converted date
    #     collection.update_one(
    #         {"_id": doc["_id"]},
    #         {"$set": {"Center Classification Date": date_obj}}
    #     )

    #     # Update all documents to convert 'Recall Initiation Date' to ISODate
    # documents1 = collection.find({"Recall Initiation Date": {"$exists": True}})
    # for doc1 in documents1:
    #     date_str1 = doc1["Recall Initiation Date"]
        
    #     # Convert string '20200413' to ISODate
    #     try:
    #         date_obj1 = datetime.strptime(date_str1, "%Y%m%d")  # Parse the string to a date object
    #     except ValueError as e:
    #         print(f"Error converting date for document ID {doc1['_id']}: {e}")
    #         continue

    #     # Update the document with the converted date
    #     collection.update_one(
    #         {"_id": doc1["_id"]},
    #         {"$set": {"Recall Initiation Date": date_obj1}}
    #     )

    # # Update all documents to convert 'Termination Date' to ISODate
    # documents2 = collection.find({"Termination Date": {"$exists": True}})
    # for doc2 in documents2:
    #     date_str2 = doc2["Termination Date"]
        
    #     # Convert string '20200413' to ISODate
    #     try:
    #         date_obj2 = datetime.strptime(date_str2, "%Y%m%d")  # Parse the string to a date object
    #     except ValueError as e:
    #         print(f"Error converting date for document ID {doc2['_id']}: {e}")
    #         continue

    #     # Update the document with the converted date
    #     collection.update_one(
    #         {"_id": doc2["_id"]},
    #         {"$set": {"Termination Date": date_obj2}}
    #     )

#print("Data inserted into MongoDB successfully.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

# client.close()

In [19]:
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import plotly.express as px
from geopy.geocoders import Nominatim
import time
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact

# Define the API endpoint and parameters
url = "https://api.fda.gov/food/enforcement.json?search=report_date:[20200101+TO+20241005]&limit=500"

# Send a GET request to the API
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    recalls = data.get('results', [])
    recall_df = pd.DataFrame(recalls)

    # clean_df = recall_df.rename(columns={
    #     "status": "Status", 
    #     "city": "City",
    #     "state": "State",
    #     "country":"Country",
    #     "classification":"Classification",
    #     "openfda":"Open FDA",
    #     "product_type":"Product Type",
    #     "event_id":"Event ID",
    #     "recalling_firm":"Recalling Firm",
    #     "address_1":"Address 1",
    #     "address_2":"Address 2",
    #     "postal_code":"Postal Code",
    #     "voluntary_mandated":"Voluntary Mandated",
    #     "initial_firm_notification":"Initial Firm Notification",
    #     "distribution_pattern":"Distribution Pattern",
    #     "recall_number":"Recall Number",
    #     "product_description":"Product Description",
    #     "product_quantity":"Product Quantity",
    #     "reason_for_recall":"Reason for Recall",
    #     "recall_initiation_date":"Recall Initiation Date",
    #     "center_classification_date":"Center Classification Date",
    #     "termination_date":"Termination Date",
    #     "report_date":"Report Date",
    #     "code_info":"Code Info",
    #     "more_code_info":"More Code Info",
    # })

    # Convert DataFrame to dictionary format for MongoDB
    recall_records = recall_df.to_dict("records")

    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Or use your MongoDB URI
    db = client['fda_recall_data3']  # Database name
    collection = db['recalls']  # Collection name

    collection.update_many({}, {"$unset": {"Openfda": "",
                                        "event_id": "",
                                        "Product_type": "",
                                        "Address_2": "",
                                        "Voluntary_mandated": "",
                                        "Initial_firm_notification": "",
                                        "Distribution_pattern": "",
                                        "Recall_number": "",
                                        "Product_description": "",
                                        "Product_quantity": "",
                                        "Reason_for_recall": "",
                                        "Report_date": "",
                                        "Code_info": "",
                                        "More_code_info": ""
                                        }}          
                            )

    # Rename the column "Recall_firm" to "Recall Firm"
    collection.update_many({}, {"$rename": {"Recalling_firm": "Recall Firm",
                                            "Address_1": "Address",
                                            "Postal_code": "Postal Code",
                                            "Recall_initiation_date": "Recall Initiation Date",
                                            "Center_classification_date": "Center Classification Date",
                                            "Termination_date": "Termination Date"
                                            }}
                            )
    
# client.close()


In [20]:
collection.find_one()